In [1]:
import sys
sys.path.append('..')


In [2]:
import numpy as np
import electrode2currentmap as e2cm
import effectivecurrent2brightness as ec2b
from scipy import interpolate
from utils import TimeSeries
import matplotlib.pyplot as plt
from itertools import product
import utils
import time
import string
import scipy
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
import skimage.io as skio

In [4]:
x = 3200
y = 2500
r = e2cm.Retina(axon_map='../retina_2500by3200_lambda4.npz',
                sampling=25, ylo=-y, yhi=y, xlo=-x, xhi=x, axon_lambda=4)

xlist=[]
ylist=[]
rlist=[]
e_spacing = 525

for x in np.arange(-2362, 2364, e_spacing):
    for y in np.arange(-1312, 1314, e_spacing):
        xlist.append(x)
        ylist.append(y)
        rlist.append(100)

e_all = e2cm.ElectrodeArray(rlist,xlist,ylist)

e_rf=[]
for e in e_all.electrodes:
    e_rf.append(e2cm.receptive_field(e, r.gridx, r.gridy, e_spacing))
    
temporal_model = ec2b.TemporalModel()
ecs, cs  = r.electrode_ecs(e_all)

fps = 30
tm1 = ec2b.TemporalModel()


In [ ]:
for letter in string.ascii_uppercase:
    eyechart = (255 - skio.imread('./letters/%s.jpg'%letter)) / 255.
    plt.matshow(eyechart, cmap='gray')
    frames = np.concatenate([eyechart[..., np.newaxis], eyechart[..., np.newaxis]],-1)
    pt=[]
    for rf in e_rf:
        rflum = e2cm.retinalmovie2electrodtimeseries(rf, frames)
        #plt.plot(rflum)
        ptrain = e2cm.Movie2Pulsetrain(rflum, amp_max=50)
        #plt.plot(ptrain.data)
        pt.append(ptrain)
    rs=1/(fps*pt[0].tsample)  
    brightness_movie = ec2b.pulse2percept(tm1, ecs, r, pt,
                                          int(rs), n_jobs=8, dojit=False, tol=.01)

    fig, ax = plt.subplots(1)
    ax.matshow(brightness_movie.data[:, :, 6], cmap='gray')
    ax.set_axis_off()
    fig.savefig("white_%s_percept.png"%letter)